In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd


pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 1000)  # Set wide enough width
pd.set_option("display.expand_frame_repr", False)  # Prevent wrapping

In [ ]:
data_folder = "../data/"
# Get a list of all CSV files in the data folder
csv_files = [file for file in os.listdir(data_folder) if file.endswith(".csv")]

# Read each CSV file into a pandas dataframe and store them in a dictionary
dataframe_dict = {file[: file.find("-")]: pd.read_csv(os.path.join(data_folder, file), header=0) for file in csv_files}
print("list of .csv files: ", list(dataframe_dict.keys()))

In [ ]:
dataframe_dict["wheat"].head(7)

In [ ]:
# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate over each dataframe in the dictionary
for key, df in dataframe_dict.items():
    # Select only the 'Date' and 'Last' columns and create a copy
    temp_df = df[["Time", "Last"]].copy()
    # Rename the 'Last' column to the name of the commodity
    temp_df.columns = ["Date", f"{key}"]

    # If combined_df is empty, assign temp_df to it (i.e., first iteration)
    if combined_df.empty:
        combined_df = temp_df
    else:
        # Merge the temp_df with combined_df on 'Date' column
        combined_df = pd.merge(combined_df, temp_df, on="Date", how="outer")

combined_df.set_index("Date", inplace=True)
combined_df.head()

In [ ]:
# Find the first non-NA/null index for each column in the combined dataframe
first_non_na_indices = combined_df.apply(lambda col: col.first_valid_index())

# Print the indices of the first non-NA/null values for each column
print(first_non_na_indices)

In [ ]:
# Interpolate missing values in the combined dataframe using linear interpolation
interpolated_df = combined_df.interpolate(method="linear")

# Select data from a specific index onwards
data_df = interpolated_df[first_non_na_indices.max() :]

In [ ]:
# Save the data_df to a CSV file
data_df.to_csv("data.csv")

In [ ]:
data_df.plot(figsize=(6, 4), title="Commodity Prices")
plt.xticks(rotation=-90)
plt.ylabel("Price")
plt.xlabel("Date")